In [1]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.4 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset

#import bitsandbytes as bnb
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

In [3]:
def print_trainable_parameters(model):
  all_params = 0
  trainable_params = 0

  for param in model.parameters():
    all_params += param.numel()

    if param.requires_grad:
      trainable_params += param.numel()

  print(f"all params : {all_params}")
  print(f"trainable params : {trainable_params}")
  print(f"t-o ratio : {100 * trainable_params / all_params}%")

In [4]:
model_name = "EleutherAI/gpt-neox-20b" #"microsoft/phi-1_5"
dataset_name = "Abirate/english_quotes"

In [5]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=nf4_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # for decoder model

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 46 files:   0%|          | 0/46 [00:00<?, ?it/s]

model-00002-of-00046.safetensors:   0%|          | 0.00/910M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [7]:
model.gradient_checkpointing_enable() # for memory usage
model = prepare_model_for_kbit_training(model)


In [10]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    # target_modules=["q_proj", "k_proj", "v_proj"], for microsoft/phi model
    target_modules=["query_key_value"], # for eleutherai gpt neox model
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [11]:
model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

all params : 10597552128
trainable params : 8650752
t-o ratio : 0.08162971878329976%


In [12]:
dataset = load_dataset(dataset_name)
dataset = dataset.map(lambda sample : tokenizer(sample["quote"]), batched=True)

README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [13]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
        num_rows: 2508
    })
})


In [14]:
trainer = Trainer(
    model=model,
    train_dataset=dataset["train"],
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="adamw_torch_4bit",#"paged_adamw_8bit",
        report_to="none", # report_to="none" to disable wandb logging
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,2.676000
2,2.063000
3,2.443200
4,1.665500
5,1.361900
6,1.339500
7,2.718900
8,2.060800
9,2.774600
10,2.104100


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=10, training_loss=2.1207505822181703, metrics={'train_runtime': 189.9906, 'train_samples_per_second': 0.211, 'train_steps_per_second': 0.053, 'total_flos': 167211775033344.0, 'train_loss': 2.1207505822181703, 'epoch': 0.01594896331738437})

In [15]:
#trainer.evaluate()
#trainer.predict()